<a href="https://colab.research.google.com/github/YSKartal/r_reference_dis/blob/main/ref_disam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("vadis/sv-ident")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.rename_column("is_variable", "label")

In [ ]:
dataset = dataset.filter(lambda x: x['lang'] == 'en')

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'variable', 'research_data', 'doc_id', 'uuid', 'lang'],
        num_rows: 1882
    })
    validation: Dataset({
        features: ['sentence', 'label', 'variable', 'research_data', 'doc_id', 'uuid', 'lang'],
        num_rows: 209
    })
})

In [ ]:
task = "ref_disam"
model_checkpoint = "distilbert-base-cased"
model_checkpoint = "bert-base-cased"
batch_size = 8

In [ ]:
from datasets import Dataset, DatasetDict, Features, Value, ClassLabel
import pandas as pd
from transformers import pipeline

tdf = pd.read_csv("train_5.tsv", sep="\t", header=0, encoding="utf8", names=["ln","id","text","c_text","label"])
vdf = pd.read_csv("test_5.tsv", sep="\t", header=0,encoding="utf8", names=["ln","id","text","c_text","label"])

class_names =  list(set(tdf["label"].to_list()))
t_lbl_ids =[int(i.replace("url","")) for i in tdf["label"]]
v_lbl_ids =[int(i.replace("url","")) for i in vdf["label"]]

tdf["c_text"] = [str(t) for t in tdf["c_text"].to_list()]
#tdf["label"] = t_lbl_ids
#vdf["label"] = v_lbl_ids

emotion_features = Features({'c_text': Value('string'), 'label': ClassLabel(names=class_names)})

tds = Dataset.from_pandas(tdf[["c_text","label"]], features= emotion_features)
vds = Dataset.from_pandas(vdf[["c_text","label"]], features= emotion_features)

In [ ]:
ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

In [ ]:
lbl_ids = list(set([int(i.replace("url","")) for i in tdf["label"]]))
lbl_text = list(set(tdf["label"].to_list()))
print(lbl_ids)
id2label = {i: "url"+str(i) for i in lbl_ids}
print(id2label)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [ ]:
ds['train']

Dataset({
    features: ['c_text', 'label'],
    num_rows: 9112
})

In [ ]:
dataset = ds

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
task_to_keys = {
    "ref_disam": ("c_text", None),
}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Excellent article in Nature on misuse of statistics: "The P value was never meant to be used the way it's used today” 


In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [ ]:
pre_tokenizer_columns = set(dataset["train"].features)
encoded_dataset = dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Columns added by tokenizer: ['token_type_ids', 'attention_mask', 'input_ids']


In [ ]:
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf

num_labels = 2274
    
# This next little bit is optional, but will give us cleaner label outputs later
# If you're using a task other than CoLA, you will probably need to change these
# to match the label names for your task!
#id2label = {0: "0", 1: "1"}
label2id = {val: key for key, val in id2label.items()}

model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, from_pt=True
)
'''model = TFAutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels, from_pt=True
)'''

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'model = TFAutoModelForSequenceClassification.from_pretrained(\n    model_checkpoint, num_labels=num_labels, from_pt=True\n)'

In [ ]:
import datasets

validation_key = (
    "validation_mismatched"
    if task == "mnli-mm"
    else "validation_matched"
    if task == "mnli"
    else "validation"
)

tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=8,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=8,
    tokenizer=tokenizer,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
encoded_dataset["validation"]

Dataset({
    features: ['c_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2278
})

In [ ]:
from transformers import create_optimizer

num_epochs = 3
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
!pip install evaluate
from evaluate import load


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.3 MB/s eta 0:00:00


In [ ]:
metric = load("f1")

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback
import numpy as np

def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    
    return metric.compute(predictions=predictions, references=labels)


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

<PrefetchDataset element_spec={'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}>

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

"""model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"YSKartal/{model_name}-finetuned-{task}"

tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./text_classification_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)"""

#callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=5,
    #callbacks=callbacks,
)

Epoch 1/5
1139/1139 [==============================] - ETA: 0s - loss: 7.7847

KeyboardInterrupt: ignored

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("YSKartal/distilbert-base-uncased-finetuned-svident")

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some layers from the model checkpoint at YSKartal/distilbert-base-uncased-finetuned-svident were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at YSKartal/distilbert-base-uncased-finetuned-svident and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# You can, of course, use your own username and model name here once you've pushed your model using the code above!
model = TFAutoModelForSequenceClassification.from_pretrained("YSKartal/distilbert-base-uncased-finetuned-svident")
tokenizer = AutoTokenizer.from_pretrained("YSKartal/distilbert-base-uncased-finetuned-svident")

Some layers from the model checkpoint at YSKartal/distilbert-base-uncased-finetuned-svident were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at YSKartal/distilbert-base-uncased-finetuned-svident and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/360 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/711k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
sentences = [
    "The ratio of ‘not favor’ increased.",
    "For the RM, we use the “centrality of religion” scale developed by Stefan Huber which is based on Charles Glock’s dimensional model of religion and claims to measure the centrality of religion in people’s life in a comparable way across different religious cultures.",
    "H2 was also supported by the results (see Table 2).",
    "Studies of citizens’ perceptions of national level phenomena can be influenced by factors that are external or indirectly related to people's experiences of the phenomenon under study, such as media representation, nature of political discourse, and the national mood in general.",
    "One of the most prominent accounts within this literature is provided by economists Alberto Alesina and Edward Glaeser’s book Fighting Poverty in the U.S. and Europe.",
    "If we add more cases to the data set, there might be more significant independent variables.",
    "Therefore, in Japan, the item on attending religious services was adapted to “visit to a temple or Shinto-shrine” in the WVS and “visit to a place of prayer and devotion” in the ISSP."

]

In [ ]:
tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

[1 1 0 0 0 0 1]


In [ ]:
!git clone https://github.com/vadis-project/sv-ident

fatal: destination path 'sv-ident' already exists and is not an empty directory.


In [ ]:
cd sv-ident

/content/sv-ident


In [ ]:
!pip --no-cache-dir install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 37.8 MB/s 
     |████████████████████████████████| 11.3 MB 51.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.6 MB/s 
     |████████████████████████████████| 90 kB 61.5 MB/s 
     |████████████████████████████████| 23.2 MB 68.1 MB/s 
     |████████████████████████████████| 78 kB 74.5 MB/s 
     |████████████████████████████████| 881.9 MB 1.2 MB/s 
     |████████████████████████████████| 2.9 MB 72.4 MB/s 
     |████████████████████████████████| 76 kB 65.5 MB/s 
     |████████████████████████████████| 235 kB 67.6 MB/s 
     |████████████████████████████████| 303 kB 83.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.7 MB/s 
     |████████████████████████████████| 1.3 MB 61.8 MB/s 
     |████████████████████████████████| 3.3 MB 57.1 MB/s 
     |████████████████████████████████| 880 kB 68.4 MB/s 
     |██████████████████████████

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
cd notebooks/variable_detection

/content/sv-ident/notebooks/variable_detection


In [ ]:
import datetime
import os
import statistics
import time
import json
from collections import defaultdict
from argparse import ArgumentParser
import jsonlines
import datasets
import torch
import random
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

In [ ]:
# === Set your hyperparameters here ===

# # Choose the dataset to use from ["en", "de"]
# lang = "en"
# assert lang in ["en", "de"]

# # Any HuggingFace model (https://huggingface.co/models) works 
# model = "bert-base-cased"

# Directory where to save different training runs
output_dir = "./runs/"

# Directory where data is stored
data_dir = "../../data/"
assert os.path.exists(data_dir)

# Cut or pad sequences to this length
max_len = 64
assert isinstance(max_len, int)

# Lowercase text (this should match the model used; i.e., when using bert-base-cased set this to True)
lowercase = False
assert isinstance(lowercase, bool)

# Other training-specific hyperparameters
gradient_accumulation_steps = 1
learning_rate = 5e-5
weight_decay = 0.0
adam_epsilon = 1e-8
max_grad_norm = 1.0
random_weights = False
batch_size = 5
epochs = 5
max_steps = -1
warmup_steps = 0
early_stop = False
eval_steps = 100
num_labels = 2
# Strategy for saving model. Possible values: ["no", "epoch", "steps"]. See https://huggingface.co/docs/transformers/v4.15.0/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments.save_strategy
save_strategy = "no"
load_best_model_at_end = True  # This can only be true if save strategy is not "no"

# Set seed for reproducibility
seed = 42

# ================ End ================

In [ ]:
def make_output_dir(output_dir):
    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime("%Y%m%d-%H%M%S")
    output_dir = os.path.join(output_dir, timestamp)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

In [ ]:
class Dataset:
    def __init__(self, dataset, lowercase=False, batch_size=16, max_len=64):
        self.dataset = dataset
        self.lowercase = lowercase
        self.batch_size = batch_size
        self.max_len = max_len

    def _encode(self, example):
        return self.tokenizer(
            example["text"],
            truncation=True,
            max_length=self.max_len,
            padding="max_length",
        )

    def format(self, dataset):
        dataset = dataset.map(self._encode, batched=True)
        try:
            dataset.set_format(
                type="torch",
                columns=["input_ids", "token_type_ids", "attention_mask", "label"],
            )
        except:
            try:
                dataset.set_format(
                type="torch",
                columns=["input_ids", "attention_mask", "label"],
            )
            except:
                raise Exception("Unable to set columns.")
        return dataset

    def format_data(self, tokenizer, batch_size=None):
        print("Formatting data...")
        self.tokenizer = tokenizer
        if batch_size:
            self.batch_size = batch_size

        self.train_dataset = self.format(self.dataset["train"])
        # self.validation_dataset = self.format(self.dataset["validation"])
        self.test_dataset = self.format(self.dataset["test"])
        print("Done formatting.")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def compute_metrics(p, average="macro"):
    pred, true = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=true, y_pred=pred)
    recall = recall_score(y_true=true, y_pred=pred, average=average)
    precision = precision_score(y_true=true, y_pred=pred, average=average)
    f1 = f1_score(y_true=true, y_pred=pred, average=average)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
    }


class TextClassifier:
    def __init__(
        self,
        model_name=None,
        lowercase=False,
        max_len=64,
        num_labels=2,
        output_dir="",
        seed=None,
    ):
        self.output_dir = output_dir

        # Model parameters
        self.model_name = model_name
        self.do_lower_case = lowercase
        self.max_len = max_len
        self.num_labels = num_labels
        
        # Set seed for reproducibility
        if seed:
            set_seed(seed)

        self.model_init()

    def model_init(self):
        print("Loading pre-trained tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)

        try:
            self.tokenizer.do_lower_case = self.do_lower_case
        except:
            try:
                self.tokenizer.do_lowercase_and_remove_accent = self.do_lower_case
            except:
                raise Exception("Unable to set value for 'do_lower_case' or 'do_lowercase_and_remove_accent'")
        self.tokenizer.model_max_length = self.max_len
        print("Done.")

        self.model_config = AutoConfig.from_pretrained(self.model_name)
        self.model_config.num_labels = self.num_labels

        print("Loading pre-trained model...")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name, config=self.model_config
        )
        print(f"Loaded model from path: {self.model_name}")

    def save_model(self, save_path=None):
        save_dir = save_path if save_path else self.output_dir
        if not os.path.isdir(save_dir):
            save_dir.mkdir(parents=True)
        torch.save(self.model, save_dir)
        print(f"Saved model to path: {save_dir}")

    def load_model(self, path):
        if os.path.exists(path):
            print("Loading local model state dict...")
            self.model.load_state_dict(torch.load(path))
            print(f"Loaded model from path: {path}")
        else:
            print(f"Model path does not exist: {path}")
            raise Exception(f"The specified file path ({path}) does not exist!")


class TextClassifierTrainer(TextClassifier):
    def __init__(
        self,
        output_dir="",
        # bfloat16=False,
        epochs=5,
        batch_size=16,
        warmup_steps=500,
        weight_decay=0.9,
        random_weights=False,
        load_best_model_at_end=False,
        early_stop=False,
        eval_steps=500,
        seed=42,
        save_strategy="no",
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.output_dir = output_dir
        self.logging_dir = os.path.join(output_dir, "logs")

        # Model parameters
        self.epochs = epochs
        self.batch_size = batch_size
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.load_best_model_at_end = load_best_model_at_end if save_strategy != "no" else False
        self.random_weights = random_weights
        self.early_stop = early_stop
        self.eval_steps = eval_steps
        self.save_strategy = save_strategy

        self.seed = seed

    def train(self, train_dataset, eval_dataset):

        ### Training
        print("Initializing trainer...")

        training_args = TrainingArguments(
            output_dir=self.output_dir,
            num_train_epochs=self.epochs,
            per_device_train_batch_size=self.batch_size,
            per_device_eval_batch_size=self.batch_size * 4,
            warmup_steps=self.warmup_steps,
            weight_decay=self.weight_decay,
            # logging_dir=self.logging_dir,
            # logging_strategy="steps",
            # logging_steps=self.eval_steps,
            evaluation_strategy="steps",
            eval_steps=self.eval_steps,
            load_best_model_at_end=self.load_best_model_at_end,
            save_strategy=self.save_strategy,
            seed=self.seed,
        )

        trainer = Trainer(
            model=self.model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
        )
        print("Trainer initialized.")
        print("Training...")
        trainer.train()
        print("Done training.")

        ### Validation
        print("Evaluating...")
        res = trainer.evaluate()
        print("Evaluation results:")
        print(
            f'Eval loss: \t{res["eval_loss"]}, Eval Acc: \t{res["eval_accuracy"]}, Eval P: \t{res["eval_precision"]}, Eval R: \t{res["eval_recall"]}, Eval F1: \t{res["eval_f1-score"]}'
        )
        print("Done evaluating.")

        if self.save_strategy != "no":
            trainer.save_model()
            trainer.save_state()
            self.trained_model_path = os.path.join(self.output_dir, "pytorch_model.bin")
            assert self.trained_model_path.is_file()
            print(f"Saved model to path: {self.output_dir}")

        return res

In [ ]:
def train(args):
    # _filename = "variable_detection_german_dev.tsv" if args.data_lang == "de" else "variable_detection_dev.tsv"
    # data_path = os.path.join(args.data_dir, _filename)
    if isinstance(args.train_data, str):
        try:
            assert os.path.exists(args.train_data)
        except:
            raise Exception(f"Failed to load: {args.train_data}")

        train_df = pd.read_csv(args.train_data, sep="\t")
        
    elif isinstance(args.train_data, pd.DataFrame):
        train_df = args.train_data
    else:
        raise Exception(f"Invalid type for {type(args.train_data)}")

    train_df.rename(columns={"is_variable": "label"}, inplace=True)

    X_idx = train_df.index.to_numpy()
    y = train_df.label.to_numpy()

    kf = KFold(n_splits=10)
    kf.get_n_splits(X_idx)

    results = defaultdict(list)

    experiment_output_dir = make_output_dir(args.output_dir)

    # Train model with cross-validation
    print("Training models with cross-validation...")
    for i, (train_index, test_index) in enumerate(tqdm(kf.split(X_idx))):
        train_dataset = datasets.Dataset.from_pandas(train_df.iloc[train_index])
        test_dataset = datasets.Dataset.from_pandas(train_df.iloc[test_index])

        run_output_dir = experiment_output_dir
        if args.save_strategy != "no":
            run_output_dir = make_output_dir(experiment_output_dir)

        trainer = TextClassifierTrainer(
            model_name=args.model,
            lowercase=args.lowercase,
            epochs=args.epochs,
            batch_size=args.batch_size,
            output_dir=run_output_dir,
            warmup_steps=args.warmup_steps,
            weight_decay=args.weight_decay,
            load_best_model_at_end=args.load_best_model_at_end,
            random_weights=args.random_weights,
            early_stop=args.early_stop,
            eval_steps=args.eval_steps,
            save_strategy=args.save_strategy,
            seed=args.seed,
        )

        dataset = Dataset({"train": train_dataset, "test": test_dataset})
        dataset.format_data(trainer.tokenizer)

        result = trainer.train(dataset.train_dataset, dataset.test_dataset)

        for k, v in result.items():
            # if k in results:
            #     results[k] += [v]
            # else:
            #     results[k] = [v]
            results[k].append(v)

    # Compute mean and standard deviation
    print("***** Cross-Validation Results *****")
    for k, v in results.items():
        skip = True
        for m in ["accuracy", "precision", "recall", "f1-score"]:
            if m in k:
                skip = False
        if skip:
            continue
        mean, std, pstd = (
            statistics.mean(v),
            statistics.stdev(v),
            statistics.pstdev(v),
        )
        print(
            k + ":\n",
            "Mean:",
            round(mean, 4),
            "\tStd.:",
            round(std, 4),
            "\tPStd:",
            round(pstd, 4),
        )
        
        results_file = os.path.join(experiment_output_dir, "results.jsonl")
        with jsonlines.open(results_file, "a") as writer:
            writer.write({k: {"Mean": mean, "Std": std, "PStd": pstd}})

    raw_results_file = os.path.join(experiment_output_dir, "raw_results.json")
    with open(raw_results_file, "w") as fp:
        json.dump(results, fp)

In [ ]:
models_en = ["bert-base-cased", "bert-base-uncased", "roberta-base", "xlnet-base-cased"]
models_de = ["bert-base-multilingual-cased", "bert-base-german-cased", "xlm-roberta-base", "xlm-mlm-ende-1024"]

for lang,models in [("en", models_en), ("de", models_de)]:
    for model in models:
        train_data = pd.concat([pd.read_csv(os.path.join(data_dir, "trial", "train", lang+".tsv"),sep="\t"), pd.read_csv(os.path.join(data_dir, "trial", "test", lang+".tsv"),sep="\t")])

        # Set hyperparameters
        class Args:
            model = model
            output_dir = os.path.join(output_dir, lang, model)
            data_dir = data_dir
            data_lang = lang
            max_len = max_len
            lowercase = lowercase
            gradient_accumulation_steps = gradient_accumulation_steps
            learning_rate = learning_rate
            weight_decay = weight_decay
            random_weights = random_weights
            adam_epsilon = adam_epsilon
            max_grad_norm = max_grad_norm
            batch_size = batch_size
            epochs = epochs
            max_steps = max_steps
            warmup_steps = warmup_steps
            early_stop = early_stop
            eval_steps = eval_steps
            num_labels = num_labels
            seed = seed
            save_strategy = save_strategy
            load_best_model_at_end = load_best_model_at_end
            train_data = train_data

        args = Args()

        # Run training
        train(args)

Training models with cross-validation...


0it [00:00, ?it/s]

Loading pre-trained tokenizer...


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

The following columns in the training set  don't have a corresponding argument in `XLMForSequenceClassification.forward` and have been ignored: doc_id, context_id, uuid, text, variable, __index_level_0__.
***** Running training *****
  Num examples = 615
  Num Epochs = 5
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 615


Trainer initialized.
Training...


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score
100,No log,0.598721,0.811594,0.405797,0.500000,0.448000
200,No log,0.509779,0.811594,0.405797,0.500000,0.448000
300,No log,0.503080,0.811594,0.405797,0.500000,0.448000


The following columns in the evaluation set  don't have a corresponding argument in `XLMForSequenceClassification.forward` and have been ignored: doc_id, context_id, uuid, text, variable, __index_level_0__.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 20
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set  don't have a corresponding argument in `XLMForSequenceClassification.forward` and have been ignored: doc_id, context_id, uuid, text, variable, __index_level_0__.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 20
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 

KeyboardInterrupt: ignored